In [1]:
#the below two lines of code are only for display purposes when using jupyter notebook. If using another IDE, you can delete the next two lines
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [27]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
import altair as alt
from altair import datum
alt.data_transformers.enable('json') 

def get_qnt(keystring, qnt_id):
    newqnt_by_cico = df[df['incident_characteristics'].str.contains(keystring)==True]
    newqnt_by_cico = newqnt_by_cico.groupby(['city_or_county', 'state_abr']).count()
    newqnt_by_cico.reset_index(inplace = True)
    newqnt_by_cico = newqnt_by_cico.drop(newqnt_by_cico.columns.difference(['city_or_county','state_abr','incident_id']), 1)
    
    ct = qnt_id + 'rpt_count'
    qnt = qnt_id + "_qnt"

    newqnt_by_cico.columns=['city_or_county', 'state_abr', ct]
    newqnt_by_cico = pd.merge(newqnt_by_cico, cico_dat, on=['city_or_county','state_abr'])
    newqnt_by_cico[qnt] = newqnt_by_cico[ct]/newqnt_by_cico['population']
    newqnt_by_cico = newqnt_by_cico.melt(['city_or_county', 'population', 'state_abr', 'rpt_qnt'], value_vars=['kill_qnt', 'inj_qnt', qnt])
    newqnt_by_cico.columns = ['qnt_type' if x=='variable' else 'aux_qnt' if x=='value' else x for x in newqnt_by_cico.columns]
    return newqnt_by_cico

print("imports completed \n")
df = pd.read_csv('stage3.csv')

print("df loaded \n")
state_abrs = pd.read_csv('state_abrs.csv')

print("state abbreviations loaded \n")
df = pd.merge(df, state_abrs, how='left', on='state')
print("df merged with state abbreviations \n")

cico_dat = pd.read_csv('cico_dat.csv')
print("city-county data loaded \n")
cico_dat['inj_qnt'] = cico_dat['n_injured']/cico_dat['population']
cico_dat['kill_qnt'] = cico_dat['n_killed']/cico_dat['population']
cico_dat['rpt_qnt'] = cico_dat['n_rpts']/cico_dat['population']

imports completed 

df loaded 

state abbreviations loaded 

df merged with state abbreviations 

city-county data loaded 



In [1]:
## DATA CLEANING ## 

#read in initial data
df = pd.read_csv('stage3.csv', dtype=object)
df = df.fillna('NO_DATA')

init_len = len(df)

# put all columns to be fixed in a list, then fix them all via for-loop
# while fixing, do other needed string processing. convert to true list type

# normalize the nested lists in the abnormal columns
abnorm_cols = [df.participant_name, df.participant_age_group,df.participant_age, df.participant_gender, df.participant_status, df.participant_type,\
               df.incident_characteristics]
col_names = ['participant_name','participant_age_group','participant_age','participant_gender','participant_type','participant_status','incident_characteristics']
for i in range(len(abnorm_cols)):
    l = hf.norm_nestedList(abnorm_cols[i].to_list())
    df = df.drop(col_names[i],1)
    df.insert(len(df.columns), col_names[i], l)
    #convert list-like strings to true lists of string objects
    if (i < (len(abnorm_cols)-1)):
        # for all but the last column, there is a digit and colon character for each delimited item which must be removed
        df[col_names[i]] = df[col_names[i]].str.replace(r'.:', '').str.split('|')
    else:
        # this column contains no offeninding digits, so just split by delimiter
        df[col_names[i]] = df[col_names[i]].str.split('|')

prim_chars = ['Non-Shooting Incident', 'Shot - Dead (murder, accidental, suicide)', 'Shot - Wounded/Injured','Shots Fired - No Injuries']
df['prim_ichars'] = df['incident_characteristics'].apply(
    lambda row: [val for val in row if val in prim_chars]
)
df['sec_ichars'] = df['incident_characteristics'].apply(
    lambda row: [val for val in row if val not in prim_chars]
)

state_dat = pd.read_csv('vega_state_ids.csv')
df = pd.merge(df, state_dat, how='left', on='state')

state_dat = pd.read_csv('state_abrs.csv')
df = pd.merge(df, state_dat, how='left', on='state')

final_len = len(df)
p = 100 - (final_len/init_len * 100)
print("Filtering process eliminated " + str(round(p, 1)) + "% of the initial data \n")

Filtering process eliminated 0.0% of the initial data 



In [85]:
col_names = ['participant_age', 'participant_gender', 'participant_type', 'participant_status']

# for many rows, the number of ages given is less than the number of characteristics given. 
# in this case, throw out the row. this is to avoid the following:
# 1. assigning an exact age to some characteristic arbitarily
# 2. forgoing using exact ages altogether.
for i in range(len(col_names)): 
    if (i > 0):
        df = df[df['participant_age'].map(len)-df[col_names[i]].map(len)==0]
for col in col_names:
    df = df[df[col]!='NO_DATA']
    
df = df.reset_index(drop=True)

# create a new table that has one row per person/participant (instead of one row per incident)
# capture incident ID so this can be merged with main dataset as needed

# create "seed table" from first row
ages     = list(df.participant_age[0])
genders  = list(df.participant_gender[0])
types    = list(df.participant_status[0])
statuses = list(df.participant_type[0])
d = {'Age':ages, 'Gender':genders, 'Type':types, 'Status':statuses}
person_tbl = pd.DataFrame(d)
person_tbl['incident_id'] = df.incident_id[0]
c = 0
for i in range(len(df)):
    if (i > 0):
        ages     = list(df.participant_age[i])
        genders  = list(df.participant_gender[i])
        types    = list(df.participant_status[i])
        statuses = list(df.participant_type[i])
        d = {'Age':ages, 'Gender':genders, 'Type':types, 'Status':statuses}
        tbl = pd.DataFrame(d)
        tbl['incident_id'] = df.incident_id[i]
        person_tbl = pd.concat([person_tbl, tbl])
        c = c + 1
        p = c/len(df)*100
        print(str(c) + " rows processed, " + str(round(p, 1)) + "% complete", end='\r')
# write to disk so the above slow-running for loop only has to be done once
person_tbl.to_csv('person_tbl.csv', index=False)

In [62]:
person_tbl = pd.read_csv('person_tbl.csv', dtype=object)
for col in person_tbl.columns:
    person_tbl = person_tbl[~person_tbl[col].str.contains("NO_DATA")]

nonsense_types = ['Injured, Unharmed', 'Killed, Injured', 'Killed, Unharmed']
for item in nonsense_types:
    person_tbl = person_tbl[~person_tbl['Status'].str.contains(item)]

person_tbl['Status'] = person_tbl['Status'].str.replace(', Arrested', '')

person_tbl = person_tbl[~person_tbl['Gender'].str.contains('Male, female')]

for col in person_tbl.columns:
    if (col == 'Gender') or (col == 'Type') or (col == 'Status'):
        person_tbl = hf.pluck_digits(person_tbl, col)

person_tbl['Age'] = person_tbl['Age'].astype(float)
person_tbl = person_tbl[(person_tbl['Age']>=1) & (person_tbl['Age']<=117) | (person_tbl['Age'] == 0)]
person_tbl['Age'] = person_tbl['Age'].astype(int)
person_tbl = pd.merge(person_tbl, df, how = 'left', on='incident_id')

to_drop = ['participant_name','participant_age','participant_age_group','participant_gender','participant_type','participant_status']

for col in person_tbl:
    if col in to_drop:
        person_tbl = person_tbl.drop(col, 1)

person_tbl

,Age,Gender,Type,Status,incident_id,date,state,city_or_county,address,n_killed,...,longitude,n_guns_involved,notes,participant_relationship,sources,state_house_district,state_senate_district,incident_characteristics,prim_ichars,sec_ichars
0,31,Male,Subject-Suspect,Unharmed,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,...,-82.1377,2,NO_DATA,NO_DATA,http://www.morningjournal.com/general-news/201...,56,13,"[Shot - Wounded/Injured, Shot - Dead (murder, ...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",[Bar/club incident - in or around establishment]
1,33,Male,Victim,Killed,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,...,-82.1377,2,NO_DATA,NO_DATA,http://www.morningjournal.com/general-news/201...,56,13,"[Shot - Wounded/Injured, Shot - Dead (murder, ...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",[Bar/club incident - in or around establishment]
2,34,Male,Victim,Injured,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,...,-82.1377,2,NO_DATA,NO_DATA,http://www.morningjournal.com/general-news/201...,56,13,"[Shot - Wounded/Injured, Shot - Dead (murder, ...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",[Bar/club incident - in or around establishment]
3,33,Male,Victim,Injured,478855,1/1/2013,Ohio,Lorain,1776 East 28th Street,1,...,-82.1377,2,NO_DATA,NO_DATA,http://www.morningjournal.com/general-news/201...,56,13,"[Shot - Wounded/Injured, Shot - Dead (murder, ...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",[Bar/club incident - in or around establishment]
4,29,Female,Victim,Killed,478925,1/5/2013,Colorado,Aurora,16000 block of East Ithaca Place,4,...,-104.802,NO_DATA,NO_DATA,NO_DATA,http://denver.cbslocal.com/2013/01/06/officer-...,40,28,"[Shot - Dead (murder, accidental, suicide), Of...","[Shot - Dead (murder, accidental, suicide)]","[Officer Involved Incident, Officer Involved S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161405,25,Female,Subject-Suspect,Unharmed,1083142,3/31/2018,Louisiana,Rayne,North Riceland Road and Highway 90,0,...,NO_DATA,1,NO_DATA,NO_DATA,http://www.klfy.com/news/local/rayne-woman-cha...,NO_DATA,NO_DATA,[Shots Fired - No Injuries],[Shots Fired - No Injuries],[]
161406,21,Male,Victim,Injured,1083151,3/31/2018,Louisiana,Gretna,1300 block of Cook Street,0,...,-90.0442,1,NO_DATA,NO_DATA,http://www.nola.com/crime/index.ssf/2018/04/sh...,85,7,[Shot - Wounded/Injured],[Shot - Wounded/Injured],[]
161407,42,Male,Victim,Killed,1082514,3/31/2018,Texas,Houston,12630 Ashford Point Dr,1,...,-95.611,1,"Vic was found shot to death in car on 4/1/18, ...",NO_DATA,http://www.khou.com/article/news/hpd-investiga...,149,17,"[Shot - Dead (murder, accidental, suicide)]","[Shot - Dead (murder, accidental, suicide)]",[]
161408,58,Female,Victim,Killed,1081940,3/31/2018,Maine,Norridgewock,434 Skowhegan Rd,2,...,-69.7691,2,"ALT: US 2, shot wife then self, handgun, shotg...",1::Significant others - current or former,https://www.centralmaine.com/2018/03/31/police...,111,3,"[Shot - Dead (murder, accidental, suicide), Su...","[Shot - Dead (murder, accidental, suicide)]","[Suicide^, Murder/Suicide, Domestic Violence]"


In [115]:
alt.Chart(person_tbl).mark_boxplot().encode(
    x = 'Type:N',
    y = 'Age:Q'
).properties(
    height = 250,
    width = 500,
    title = 'Age Distributions for Participant Types in Gun Violence Incidents'
)

alt.Chart(...)

In [64]:
alt.Chart(person_tbl).mark_boxplot().encode(
    x = 'Status:N',
    y = 'Age:Q'
).properties(
    height = 250,
    width = 400,
    title = 'Age Distributions for Participant Statuses in Gun Violence Incidents'
)

alt.Chart(...)

In [63]:
alt.Chart(person_tbl).mark_boxplot().encode(
    x = 'Gender:N',
    y = 'Age:Q'
).properties(
    height = 250,
    width = 200,
    title = 'Age Distributions by Participant Gender in Gun Violence Incidents'
)

alt.Chart(...)

In [124]:
alt.Chart(person_tbl).mark_bar().encode(
    alt.X("Age:Q",
    bin=alt.Bin(extent=[0, 120], step=10),
    scale = alt.Scale(domain=(0,100))
    ),
    y = 'count():Q'
)

alt.Chart(...)

In [125]:
ages = person_tbl['Age'].to_list()
hf.normaltestPrint(ages, 'Ages of Gun Crime Participants')


Ages of Gun Crime Participants--- Dataset Stats:
Size of Ages of Gun Crime Participants Dataset = 161410
Normality Test-->

p value = 0.0

We reject the null hypothesis that the distribution of Ages in the 'Ages of Gun Crime Participants' dataset is normal.


In [158]:
alt.Chart(person_tbl).mark_bar().encode(
    x = alt.X('state:N', sort='-y'),
    y = 'count():Q',
    color = 'Type:N'
).properties(
    title = 'Distribution of Female Victims and Suspects in Gun Crime, by State'
).transform_filter(
    (datum['Gender'] == 'Female')
)

alt.Chart(...)

In [165]:
person_tbl.dtypes

Age                             int32
Gender                         object
Type                           object
Status                         object
incident_id                    object
date                           object
state                          object
city_or_county                 object
address                        object
n_killed                       object
n_injured                      object
incident_url                   object
source_url                     object
incident_url_fields_missing    object
congressional_district         object
gun_stolen                     object
gun_type                       object
latitude                       object
location_description           object
longitude                      object
n_guns_involved                object
notes                          object
participant_relationship       object
sources                        object
state_house_district           object
state_senate_district          object
incident_cha

In [184]:
ave_age_by_state = person_tbl.groupby('state')['Age'].mean()
ave_age_by_state = ave_age_by_state.reset_index()
ave_age_by_state.dtypes
ave_age_by_state.columns = ['AveAge' if x == 'Age' else x for x in ave_age_by_state.columns]
ave_age_by_state = ave_age_by_state.sort_values(by='AveAge', ascending = False)
ave_age_by_state.reset_index(inplace = True, drop=True)
ave_age_by_state

,state,AveAge
0,Montana,37.680751
1,Wyoming,36.943005
2,Hawaii,35.895105
3,Vermont,35.489231
4,Maine,34.891213
5,Idaho,34.239051
6,West Virginia,33.720299
7,Nevada,33.591154
8,Oregon,32.933482
9,Arizona,32.900804


In [185]:
state_sort = ave_age_by_state.state.to_list()
alt.Chart(person_tbl).mark_boxplot().encode(
    x = alt.X('state:N', sort=state_sort),
    y = 'Age:Q'
).properties(
    title = 'Distribution of Ages for Gun Crime Suspects, by State'
).transform_filter(
    (datum['Type'] == 'Subject-Suspect')
)

alt.Chart(...)

In [186]:
state_sort = ave_age_by_state.state.to_list()
alt.Chart(person_tbl).mark_boxplot().encode(
    x = alt.X('state:N', sort=state_sort),
    y = 'Age:Q'
).properties(
    title = 'Distribution of Ages for Gun Crime Suspects, by State'
).transform_filter(
    (datum['Type'] == 'Victim')
)

alt.Chart(...)

In [60]:
sets = df.sec_ichars
sets = [val if val!='[]' else val for val in sets]
te = TransactionEncoder()
te_ary = te.fit(sets).transform(sets)
fi_df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(fi_df, min_support=0.01, use_colnames=True)
# alternatively:
# frequent_itemsets = apriori(asc_df, min_support=0.01, use_colnames=True)
# frequent_itemsets = fpmax(asc_df, min_support=0.01, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False).reset_index(drop=True)
frequent_itemsets

,support,itemsets
0,0.149662,(Possession (gun(s) found during commission of...
1,0.094848,(ATF/LE Confiscation/Raid/Arrest)
2,0.085282,(Officer Involved Incident)
3,0.084660,(Possession of gun by felon or prohibited person)
4,0.081762,(Drug involvement)
...,...,...
66,0.010496,(Home Invasion - No death or injury)
67,0.010496,"(Home Invasion - No death or injury, Home Inva..."
68,0.010466,(Self-Inflicted (not suicide or suicide attemp...
69,0.010466,(Self-Inflicted (not suicide or suicide attemp...


In [61]:
from mlxtend.frequent_patterns import association_rules

asc_tbl = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.0)
asc_tbl = asc_tbl.sort_values(by='lift', ascending=False).reset_index(drop=True)
asc_tbl.to_csv('asc_tbl.csv',index=False)

In [36]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.0)
rules = rules.sort_values(by='lift', ascending = False).reset_index(drop=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(School Incident),"(Gun at school, no death/injury - elementary/s...",0.014526,0.010970,0.010970,0.755202,68.842921,0.010811,4.040188
1,"(Gun at school, no death/injury - elementary/s...",(School Incident),0.010970,0.014526,0.010970,1.000000,68.842921,0.010811,inf
2,"(Officer Involved Incident, Shots Fired - No I...","(Officer Involved Shooting - Shots fired, no i...",0.018443,0.013779,0.013275,0.719794,52.238184,0.013021,3.519632
3,"(Officer Involved Shooting - Shots fired, no i...","(Officer Involved Incident, Shots Fired - No I...",0.013779,0.018443,0.013275,0.963441,52.238184,0.013021,26.848465
4,(Home Invasion),(Home Invasion - No death or injury),0.020595,0.010496,0.010496,0.509640,48.556547,0.010280,2.017915
...,...,...,...,...,...,...,...,...,...
459,(Possession (gun(s) found during commission of...,(Shots Fired - No Injuries),0.149662,0.170031,0.012292,0.082129,0.483023,-0.013156,0.904233
460,(Shot - Wounded/Injured),(Possession (gun(s) found during commission of...,0.342071,0.149662,0.014390,0.042066,0.281073,-0.036805,0.887679
461,(Possession (gun(s) found during commission of...,(Shot - Wounded/Injured),0.149662,0.342071,0.014390,0.096147,0.281073,-0.036805,0.727916
462,(Shot - Wounded/Injured),"(Shot - Dead (murder, accidental, suicide))",0.342071,0.236218,0.020357,0.059512,0.251939,-0.060446,0.812113


In [196]:
del gBy_state
df['n_killed'] = df['n_killed'].astype(int)
df['n_injured'] = df['n_injured'].astype(int)

ctrecs_by_state = df.groupby(['state','state_pop','vega_state_id']).count()
ctrecs_by_state.reset_index(inplace=True)
ctrecs_by_state.drop(ctrecs_by_state.columns.difference(['state','state_pop', 'incident_id']), 1, inplace=True)
ctrecs_by_state.columns = ['state','state_pop', 'report_count']
ctrecs_by_state['rpt_qnt'] = ctrecs_by_state['report_count']/ctrecs_by_state['state_pop']

sumkilled_by_state = df.groupby(['state', 'state_pop','vega_state_id'])['n_killed'].sum()
sumkilled_by_state = sumkilled_by_state.reset_index()
sumkilled_by_state['kill_qnt'] = sumkilled_by_state['n_killed']/sumkilled_by_state['state_pop']

suminj_by_state = df.groupby(['state', 'state_pop'])['n_injured'].sum()
suminj_by_state = suminj_by_state.reset_index()
suminj_by_state['inj_qnt'] = suminj_by_state['n_injured']/suminj_by_state['state_pop']

gBy_state = pd.merge(ctrecs_by_state, sumkilled_by_state, on=['state','state_pop'])
gBy_state = pd.merge(gBy_state, suminj_by_state, on=['state','state_pop'])
# gBy_state = pd.merge(ctrecs_by_state, sumkilled_by_state, on=['state','state_pop'])
gBy_state.reset_index(inplace=True, drop=True)
gBy_state = gBy_state.melt(['state', 'state_pop','vega_state_id'], value_vars=['rpt_qnt', 'kill_qnt', 'inj_qnt'])
# gBy_state = gBy_state.melt(['state', 'state_pop','vega_state_id'], value_vars=['rpt_qnt', 'kill_qnt'])

gBy_state.columns = ['state', 'state_pop','vega_state_id', 'qnt_type', 'pop_quotient']

alt.Chart(gBy_state).mark_bar().encode(
    x = 'qnt_type:N',
    y = 'sum(pop_quotient):Q',
    column = 'state:N'
)

alt.Chart(...)

In [200]:
kqs = gBy_state[gBy_state['qnt_type']=='kill_qnt']
kqs = kqs.sort_values(by='pop_quotient', ascending=False)
kqs.reset_index(inplace=True, drop=False)

rqs = gBy_state[gBy_state['qnt_type']=='rpt_qnt']
rqs = rqs.sort_values(by='pop_quotient', ascending=False)
rqs.reset_index(inplace=True, drop=False)

iqs = gBy_state[gBy_state['qnt_type']=='inj_qnt']
iqs = iqs.sort_values(by='pop_quotient', ascending=False)
iqs.reset_index(inplace=True, drop=False)

rpt_sort = list(rqs.state)
kill_sort = list(kqs.state)
inj_sort = list(iqs.state)

alt.Chart(gBy_state).mark_bar().encode(
    alt.X('state:N', sort = rpt_sort),
    y = 'pop_quotient:Q',
).properties(
    title='Report-Count-over-Population Quotient by US State'
).transform_filter(
    (datum['qnt_type']=='rpt_qnt')
)

alt.Chart(...)

In [201]:
alt.Chart(gBy_state).mark_bar().encode(
    alt.X('state:N', sort = kill_sort),
    y = 'pop_quotient:Q',
).properties(
    title='Kill-Count-over-Population Quotient by US State'
).transform_filter(
    (datum['qnt_type']=='kill_qnt')
)

alt.Chart(...)

In [204]:
alt.Chart(gBy_state).mark_bar().encode(
    alt.X('state:N', sort = inj_sort),
    y = 'pop_quotient:Q',
).properties(
    title='Injury-Count-over-Population Quotient by US State'
).transform_filter(
    (datum['qnt_type']=='inj_qnt')
)

alt.Chart(...)

In [6]:
import geonamescache

cico_pop = []
cicos = []
states = []

gc = geonamescache.GeonamesCache()
c = 0

for i in range(len(cico_dat['city_or_county'])):
    search_results = gc.get_cities_by_name(cico_dat['city_or_county'][i])
    for hits in search_results:
        for key in hits.keys():
            if (hits.get(key, {}).get('countrycode') == 'US') and (hits.get(key, {}).get('admin1code') == cico_dat['state_abr'][i]):
                cico_pop.append(hits.get(key, {}).get('population'))
                states.append(hits.get(key, {}).get('admin1code'))
                cicos.append(cico_dat['city_or_county'][i])
    c = c + 1
    p = c/len(cico_dat)*100
    print(str(c) + " rows processed. " + str(round(p, 1)) + "% complete", end='\r')
d = {'city_or_county' : cicos, 'state_abr' : states, 'population' : cico_pop}
cico_dat = pd.DataFrame(d)
cico_dat.to_csv('cico_dat.csv', index=False)
cico_dat

1196 rows processed. 45.5% complete

2087 rows processed. 79.4% complete

,city_or_county,state_abr,population
0,Aberdeen,MD,15580
1,Aberdeen,SD,28102
2,Aberdeen,WA,16276
3,Abilene,TX,121721
4,Abington,MA,15985
...,...,...,...
2625,Yuma,AZ,94139
2626,Zachary,LA,16448
2627,Zanesville,OH,25498
2628,Zion,IL,24117


In [3]:
rpts_cico_dat = df.groupby(['city_or_county', 'state_abr']).count()
rpts_cico_dat.reset_index(inplace=True)
rpts_cico_dat = rpts_cico_dat.drop(rpts_cico_dat.columns.difference(['city_or_county', 'state_abr', 'incident_id']), 1)
rpts_cico_dat.columns = ['city_or_county', 'state_abr','n_rpts']

df['n_injured'] = df['n_injured'].astype(int)
injs_cico_dat =df.groupby(['city_or_county', 'state_abr'])['n_injured'].sum()
injs_cico_dat = injs_cico_dat.reset_index()

df['n_killed'] = df['n_killed'].astype(int)
kills_cico_dat = df.groupby(['city_or_county', 'state_abr'])['n_killed'].sum()
kills_cico_dat = kills_cico_dat.reset_index()

cico_dat = pd.merge(rpts_cico_dat, injs_cico_dat, on=['city_or_county', 'state_abr'])
cico_dat = pd.merge(cico_dat, kills_cico_dat, on=['city_or_county', 'state_abr'])

cico_dat = pd.merge(cico_dat, injs_cico_dat, on=['city_or_county', 'state_abr'])
cico_dat = pd.merge(cico_dat, rpts_cico_dat, on=['city_or_county', 'state_abr'])
cico_dat = pd.merge(cico_dat, kills_cico_dat, on=['city_or_county', 'state_abr'])
cico_dat.to_csv('cico_dat.csv', index=False)
cico_dat = cico_dat.melt(['city_or_county', 'population', 'state_abr', 'rpt_qnt'], value_vars=['kill_qnt', 'inj_qnt'])
cico_dat.columns = ['city_or_county', 'population', 'state_abr', 'rpt_qnt', 'qnt_type', 'aux_qnt']
cico_dat

In [42]:
alt.Chart(cico_dat).mark_circle().encode(
    x = 'rpt_qnt:Q',
    y = 'aux_qnt:Q',
    tooltip = ['city_or_county:N','state_abr:N', 'population:Q']
).properties(
    width = 500,
    height = 250,
    title = 'Injured Qnt by Report Qnt of US City/County'
).transform_filter(
    (datum['qnt_type'] == 'inj_qnt')
)

alt.Chart(...)

In [43]:
alt.Chart(cico_dat).mark_circle().encode(
    x = 'rpt_qnt:Q',
    y = 'aux_qnt:Q',
    tooltip = ['city_or_county:N','state_abr:N', 'population:Q']
).properties(
    width = 500,
    height = 250,
    title = 'Killed Qnt by Report Qnt of US City/County'
).transform_filter(
    (datum['qnt_type'] == 'kill_qnt')
)

alt.Chart(...)

In [22]:
scde_by_cico = get_qnt('Suicide', 'scde')
alt.Chart(scde_by_cico).mark_circle().encode(
    x = 'rpt_qnt:Q',
    y = 'aux_qnt:Q',
    tooltip = ['city_or_county:N','state_abr:N', 'population:Q']
).properties(
    width = 500,
    height = 250,
    title = 'Suicide Qnt by Report Qnt of US City/County'
).transform_filter(
    (datum['qnt_type'] == 'scde_qnt')
)

1288 at line 26
1288 at line 29


alt.Chart(...)

In [23]:
ofin_by_cico = get_qnt('Officer', 'ofin')

alt.Chart(ofin_by_cico).mark_circle().encode(
    x = 'rpt_qnt:Q',
    y = 'aux_qnt:Q',
    tooltip = ['city_or_county:N','state_abr:N', 'population:Q']
).properties(
    width = 500,
    height = 250,
    title = 'Officer-Involced Qnt by Report Qnt of US City/County'
).transform_filter(
    (datum['qnt_type'] == 'ofin_qnt')
)

1799 at line 26
1799 at line 29


alt.Chart(...)

In [24]:
acc_cico = get_qnt('Accident', 'acc')

alt.Chart(acc_cico).mark_circle().encode(
    x = 'rpt_qnt:Q',
    y = 'aux_qnt:Q',
    tooltip = ['city_or_county:N','state_abr:N', 'population:Q']
).properties(
    width = 500,
    height = 250,
    title = 'Accident Qnt by Report Qnt of US City/County'
).transform_filter(
    (datum['qnt_type'] == 'acc_qnt')
)

1276 at line 26
1276 at line 29


alt.Chart(...)

In [26]:
dmvi_cico = get_qnt('Domestic Violence', 'dmvi')

alt.Chart(dmvi_cico).mark_circle().encode(
    x = 'rpt_qnt:Q',
    y = 'aux_qnt:Q',
    tooltip = ['city_or_county:N','state_abr:N', 'population:Q']
).properties(
    width = 500,
    height = 250,
    title = 'Domestic-Violence Qnt by Report Qnt of US City/County'
).transform_filter(
    (datum['qnt_type'] == 'dmvi_qnt')
)

1446 at line 26
1446 at line 29


alt.Chart(...)

In [28]:
gang_by_cico = get_qnt('Gang', 'gang')

alt.Chart(gang_by_cico).mark_circle().encode(
    x = 'rpt_qnt:Q',
    y = 'aux_qnt:Q',
    tooltip = ['city_or_county:N','state_abr:N', 'population:Q']
).properties(
    width = 500,
    height = 250,
    title = 'Gang Qnt by Report Qnt of US City/County'
).transform_filter(
    (datum['qnt_type'] == 'gang_qnt')
)

alt.Chart(...)

In [37]:
mssh_by_cico = get_qnt('Mass Shoot', 'mssh')

alt.Chart(mssh_by_cico).mark_circle().encode(
    x = 'rpt_qnt:Q',
    y = 'aux_qnt:Q',
    tooltip = ['city_or_county:N','state_abr:N', 'population:Q']
).properties(
    width = 500,
    height = 250,
    title = 'Mass-Shooting Qnt by Report Qnt of US City/County'
).transform_filter(
    (datum['qnt_type'] == 'mssh_qnt')
)

alt.Chart(...)

In [40]:
arb_by_cico = get_qnt('Armed rob', 'arb')

alt.Chart(arb_by_cico).mark_circle().encode(
    x = 'rpt_qnt:Q',
    y = 'aux_qnt:Q',
    tooltip = ['city_or_county:N','state_abr:N', 'population:Q']
).properties(
    width = 500,
    height = 250,
    title = 'Armed Robbery Qnt by Report Qnt of US City/County'
).transform_filter(
    (datum['qnt_type'] == 'arb_qnt')
)

alt.Chart(...)

In [42]:
hoiv_by_cico = get_qnt('Home Invasion', 'hoiv')

alt.Chart(hoiv_by_cico).mark_circle().encode(
    x = 'rpt_qnt:Q',
    y = 'aux_qnt:Q',
    tooltip = ['city_or_county:N','state_abr:N', 'population:Q']
).properties(
    width = 500,
    height = 250,
    title = 'Home Invasion Qnt by Report Qnt of US City/County'
).transform_filter(
    (datum['qnt_type'] == 'hoiv_qnt')
)

alt.Chart(...)

In [45]:
dby_by_cico = get_qnt('Drive-by', 'dby')

alt.Chart(dby_by_cico).mark_circle().encode(
    x = 'rpt_qnt:Q',
    y = 'aux_qnt:Q',
    tooltip = ['city_or_county:N','state_abr:N', 'population:Q']
).properties(
    width = 500,
    height = 250,
    title = 'Drive-by Qnt by Report Qnt of US City/County'
).transform_filter(
    (datum['qnt_type'] == 'dby_qnt')
)

alt.Chart(...)

In [44]:
drug_by_cico = get_qnt('Drug', 'drug')

alt.Chart(drug_by_cico).mark_circle().encode(
    x = 'rpt_qnt:Q',
    y = 'aux_qnt:Q',
    tooltip = ['city_or_county:N','state_abr:N', 'population:Q']
).properties(
    width = 500,
    height = 250,
    title = 'Drug-Related Qnt by Report Qnt of US City/County'
).transform_filter(
    (datum['qnt_type'] == 'drug_qnt')
)

alt.Chart(...)